In [52]:
import numpy as np
import pandas as pd
import requests
import time
from scipy.stats import linregress
from pprint import pprint
import json
from census import Census
from config import (census_key, gkey)
import gmaps

# Census API Key
c = Census(census_key)

### Census Information API on poverty and race per zipcode

In [53]:
# Run Census Search to retrieve data on all zip codes (latest available Census)
# See: https://github.com/CommerceDataService/census-wrapper for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels
census_data = c.acs5.get(("B01003_001E", "B17001_002E", "B19013_001E", "B02001_002E", "B02001_003E", "B02001_004E", "B02001_005E", "B02001_006E", "B02001_008E", "B03001_003E"
                         ), {'for': 'zip code tabulation area:*'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B17001_002E": "Poverty Count",
                                      "B19013_001E": "Median Income",
                                      "B02001_002E": "Caucasian",
                                      "B02001_003E": "African American",
                                      "B02001_004E": "American Indian",
                                      "B02001_005E": "Asian",
                                      "B02001_006E": "Native Hawaiian",
                                      "B02001_008E": "Two or more races",
                                      "B03001_003E": "Hispanic",
                                      "zip code tabulation area": "Zipcode"})

In [54]:
census_pd.sample(5)

,Population,Poverty Count,Median Income,Caucasian,African American,American Indian,Asian,Native Hawaiian,Two or more races,Hispanic,state,Zipcode
31290,1222.0,152.0,65000.0,1047.0,0.0,26.0,16.0,0.0,12.0,447.0,06,95323
10803,25854.0,897.0,119518.0,19716.0,1848.0,26.0,2108.0,42.0,1598.0,10843.0,12,33331
604,200.0,17.0,-666666666.0,200.0,0.0,0.0,0.0,0.0,0.0,0.0,25,02641
22437,8005.0,1994.0,40668.0,7776.0,28.0,33.0,0.0,0.0,168.0,210.0,29,64759
21022,1001.0,196.0,45714.0,969.0,0.0,0.0,12.0,0.0,10.0,10.0,17,61441


In [55]:
census_pd["Poverty Rate"] = 100 * census_pd["Poverty Count"]/ census_pd["Population"]

In [56]:
census_pd.sample(5)

,Population,Poverty Count,Median Income,Caucasian,African American,American Indian,Asian,Native Hawaiian,Two or more races,Hispanic,state,Zipcode,Poverty Rate
20381,3726.0,1179.0,43113.0,2179.0,1.0,1169.0,22.0,33.0,319.0,238.0,30,59865,31.642512
28092,15519.0,2817.0,41195.0,12701.0,1003.0,122.0,365.0,23.0,960.0,1904.0,08,80903,18.151943
15036,1477.0,63.0,62201.0,1477.0,0.0,0.0,0.0,0.0,0.0,7.0,18,46165,4.265403
25789,241.0,66.0,36250.0,137.0,5.0,86.0,0.0,0.0,13.0,4.0,40,74830,27.385892
5942,8882.0,405.0,81917.0,7863.0,474.0,11.0,249.0,0.0,87.0,596.0,42,19033,4.559784


In [57]:
census_pd.columns = census_pd.columns.str.replace(' ', '_').str.lower()
census_pd.tail(5)

,population,poverty_count,median_income,caucasian,african_american,american_indian,asian,native_hawaiian,two_or_more_races,hispanic,state,zipcode,poverty_rate
33115,15.0,0.0,-666666666.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,02,99923,0.000000
33116,927.0,172.0,57375.0,446.0,3.0,366.0,0.0,7.0,105.0,26.0,02,99925,18.554477
33117,1635.0,235.0,53409.0,239.0,0.0,1210.0,22.0,0.0,147.0,116.0,02,99926,14.373089
33118,38.0,28.0,-666666666.0,38.0,0.0,0.0,0.0,0.0,0.0,0.0,02,99927,73.684211
33119,2484.0,192.0,57583.0,1670.0,2.0,496.0,69.0,9.0,208.0,121.0,02,99929,7.729469


In [58]:
census_pd["caucasian_%"] = 100 * census_pd["caucasian"]/ census_pd["population"]
census_pd["african_american_%"] = 100 * census_pd["african_american"]/ census_pd["population"]
census_pd["american_indian_%"] = 100 * census_pd["american_indian"]/ census_pd["population"]
census_pd["asian_%"] = 100 * census_pd["asian"]/ census_pd["population"]
census_pd["native_hawaiian_%"] = 100 * census_pd["native_hawaiian"]/ census_pd["population"]
census_pd["two_or_more_races_%"] = 100 * census_pd["two_or_more_races"]/ census_pd["population"]
census_pd["hispanic_%"] = 100 * census_pd["hispanic"]/ census_pd["population"]
clean_census_df=census_pd[["zipcode", "population", "poverty_rate", "median_income", "caucasian_%", "african_american_%", "american_indian_%", "asian_%", "native_hawaiian_%", "two_or_more_races_%", "hispanic_%"]]

In [59]:
clean_census_df = clean_census_df.rename(columns={'zipcode': 'zip_code'})
clean_census_df['zip_code'].apply(type).value_counts()

<class 'str'>    33120
Name: zip_code, dtype: int64

In [61]:
clean_census_df

,zip_code,population,poverty_rate,median_income,caucasian_%,african_american_%,american_indian_%,asian_%,native_hawaiian_%,two_or_more_races_%,hispanic_%
0,00601,17242.0,62.475351,13092.0,75.548080,0.840970,0.144995,0.017399,0.000000,0.661176,99.663612
1,00602,38442.0,51.014515,16358.0,79.415743,2.783414,0.000000,0.000000,0.000000,13.794808,93.452474
2,00603,48814.0,49.856599,16603.0,76.473962,3.953784,0.235588,0.745688,0.020486,2.167411,97.412628
3,00606,6437.0,64.672984,12832.0,40.810937,2.314743,0.279633,0.000000,0.000000,1.941898,99.844648
4,00610,27073.0,43.305138,19309.0,75.540206,2.570827,0.000000,0.000000,0.003694,11.280612,96.195472
...,...,...,...,...,...,...,...,...,...,...,...
33115,99923,15.0,0.000000,-666666666.0,100.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
33116,99925,927.0,18.554477,57375.0,48.112190,0.323625,39.482201,0.000000,0.755124,11.326861,2.804746
33117,99926,1635.0,14.373089,53409.0,14.617737,0.000000,74.006116,1.345566,0.000000,8.990826,7.094801
33118,99927,38.0,73.684211,-666666666.0,100.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [62]:
clean_census_df['median_income'].apply(type).value_counts()

<class 'float'>    33120
Name: median_income, dtype: int64

In [63]:
(clean_census_df.median_income <= 0).value_counts()

False    30926
True      2194
Name: median_income, dtype: int64

In [64]:
updated_census_df = clean_census_df[(clean_census_df['median_income']>0)]
updated_census_df

,zip_code,population,poverty_rate,median_income,caucasian_%,african_american_%,american_indian_%,asian_%,native_hawaiian_%,two_or_more_races_%,hispanic_%
0,00601,17242.0,62.475351,13092.0,75.548080,0.840970,0.144995,0.017399,0.000000,0.661176,99.663612
1,00602,38442.0,51.014515,16358.0,79.415743,2.783414,0.000000,0.000000,0.000000,13.794808,93.452474
2,00603,48814.0,49.856599,16603.0,76.473962,3.953784,0.235588,0.745688,0.020486,2.167411,97.412628
3,00606,6437.0,64.672984,12832.0,40.810937,2.314743,0.279633,0.000000,0.000000,1.941898,99.844648
4,00610,27073.0,43.305138,19309.0,75.540206,2.570827,0.000000,0.000000,0.003694,11.280612,96.195472
...,...,...,...,...,...,...,...,...,...,...,...
33113,99921,1986.0,14.400806,58571.0,68.026183,0.553877,15.508560,1.963746,0.000000,13.595166,4.028197
33114,99922,330.0,39.090909,34028.0,12.424242,1.212121,71.818182,0.000000,8.787879,5.757576,0.000000
33116,99925,927.0,18.554477,57375.0,48.112190,0.323625,39.482201,0.000000,0.755124,11.326861,2.804746
33117,99926,1635.0,14.373089,53409.0,14.617737,0.000000,74.006116,1.345566,0.000000,8.990826,7.094801


In [65]:
final_census_df=updated_census_df[["zip_code", "population", "poverty_rate", "median_income"]]
final_census_df

,zip_code,population,poverty_rate,median_income
0,00601,17242.0,62.475351,13092.0
1,00602,38442.0,51.014515,16358.0
2,00603,48814.0,49.856599,16603.0
3,00606,6437.0,64.672984,12832.0
4,00610,27073.0,43.305138,19309.0
...,...,...,...,...
33113,99921,1986.0,14.400806,58571.0
33114,99922,330.0,39.090909,34028.0
33116,99925,927.0,18.554477,57375.0
33117,99926,1635.0,14.373089,53409.0


In [70]:
final_census_df.to_csv("output_data/final_census_df.csv", index=False)

### CSV uploaded on Value of Care

In [17]:
value_df = pd.read_csv("data/Payment_and_Value_of_Care-Hospital.csv", low_memory=False)
value_df

,Facility ID,Facility Name,Address,City,State,ZIP Code,County Name,Phone Number,Payment Measure ID,Payment Measure Name,...,Payment,Lower Estimate,Higher Estimate,Payment Footnote,Value of Care Display ID,Value of Care Display Name,Value of Care Category,Value of Care Footnote,Start Date,End Date
0,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_30_AMI,Payment for heart attack patients,...,"$24,934","$23,490","$26,429",NaN,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment,NaN,07/01/2016,06/30/2019
1,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_30_HF,Payment for heart failure patients,...,"$17,854","$17,061","$18,676",NaN,MORT_PAYM_30_HF,Value of Care Heart Failure measur,Average Mortality and Average Payment,NaN,07/01/2016,06/30/2019
2,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_30_PN,Payment for pneumonia patients,...,"$20,216","$19,186","$21,271",NaN,MORT_PAYM_30_PN,Value of Care Pneumonia measure,Average Mortality and Higher Payment,NaN,07/01/2016,06/30/2019
3,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_90_HIP_KNEE,Payment for hip/knee replacement patients,...,"$22,216","$21,191","$23,313",NaN,COMP_PAYM_90_HIP_KNEE,Value of Care hip/knee replacement,Average Complications and Higher Payment,NaN,04/01/2016,03/31/2019
4,10005,MARSHALL MEDICAL CENTERS,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,MARSHALL,(256) 593-8310,PAYM_30_AMI,Payment for heart attack patients,...,"$24,742","$22,016","$27,967",NaN,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment,NaN,07/01/2016,06/30/2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18835,670136,BAYLOR SCOTT & WHITE MEDICAL CENTER- AUSTIN,5245 W US 290,AUSTIN,TX,78735,TRAVIS,(512) 654-2100,PAYM_90_HIP_KNEE,Payment for hip/knee replacement patients,...,Not Available,Not Available,Not Available,5.0,COMP_PAYM_90_HIP_KNEE,Value of Care hip/knee replacement,Not Available,5.0,04/01/2016,03/31/2019
18836,670143,ASCENSION SETON BASTROP,"630 HIGWAY 71 W,",BASTROP,TX,78602,BASTROP,(737) 881-7400,PAYM_30_AMI,Payment for heart attack patients,...,Not Available,Not Available,Not Available,19.0,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Not Available,19.0,07/01/2016,06/30/2019
18837,670143,ASCENSION SETON BASTROP,"630 HIGWAY 71 W,",BASTROP,TX,78602,BASTROP,(737) 881-7400,PAYM_30_HF,Payment for heart failure patients,...,Not Available,Not Available,Not Available,19.0,MORT_PAYM_30_HF,Value of Care Heart Failure measur,Not Available,19.0,07/01/2016,06/30/2019
18838,670143,ASCENSION SETON BASTROP,"630 HIGWAY 71 W,",BASTROP,TX,78602,BASTROP,(737) 881-7400,PAYM_30_PN,Payment for pneumonia patients,...,Not Available,Not Available,Not Available,19.0,MORT_PAYM_30_PN,Value of Care Pneumonia measure,Not Available,19.0,07/01/2016,06/30/2019


In [18]:
value_df.columns = value_df.columns.str.replace(' ', '_').str.lower()
value_df.head()

,facility_id,facility_name,address,city,state,zip_code,county_name,phone_number,payment_measure_id,payment_measure_name,...,payment,lower_estimate,higher_estimate,payment_footnote,value_of_care_display_id,value_of_care_display_name,value_of_care_category,value_of_care_footnote,start_date,end_date
0,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_30_AMI,Payment for heart attack patients,...,"$24,934","$23,490","$26,429",NaN,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment,NaN,07/01/2016,06/30/2019
1,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_30_HF,Payment for heart failure patients,...,"$17,854","$17,061","$18,676",NaN,MORT_PAYM_30_HF,Value of Care Heart Failure measur,Average Mortality and Average Payment,NaN,07/01/2016,06/30/2019
2,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_30_PN,Payment for pneumonia patients,...,"$20,216","$19,186","$21,271",NaN,MORT_PAYM_30_PN,Value of Care Pneumonia measure,Average Mortality and Higher Payment,NaN,07/01/2016,06/30/2019
3,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_90_HIP_KNEE,Payment for hip/knee replacement patients,...,"$22,216","$21,191","$23,313",NaN,COMP_PAYM_90_HIP_KNEE,Value of Care hip/knee replacement,Average Complications and Higher Payment,NaN,04/01/2016,03/31/2019
4,10005,MARSHALL MEDICAL CENTERS,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,MARSHALL,(256) 593-8310,PAYM_30_AMI,Payment for heart attack patients,...,"$24,742","$22,016","$27,967",NaN,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment,NaN,07/01/2016,06/30/2019


In [19]:
for column in value_df.columns:
    print(column)

facility_id
facility_name
address
city
state
zip_code
county_name
phone_number
payment_measure_id
payment_measure_name
payment_category
denominator
payment
lower_estimate
higher_estimate
payment_footnote
value_of_care_display_id
value_of_care_display_name
value_of_care_category
value_of_care_footnote
start_date
end_date


In [20]:
updated_df=value_df.drop(['county_name', 'phone_number', 'payment_footnote', 'value_of_care_footnote', 'lower_estimate', 'higher_estimate', 'start_date', 'end_date', 'payment_measure_id'], axis=1).reset_index(drop=True)
updated_df

,facility_id,facility_name,address,city,state,zip_code,payment_measure_name,payment_category,denominator,payment,value_of_care_display_id,value_of_care_display_name,value_of_care_category
0,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Payment for heart attack patients,No Different Than the National Average Payment,574,"$24,934",MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment
1,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Payment for heart failure patients,No Different Than the National Average Payment,823,"$17,854",MORT_PAYM_30_HF,Value of Care Heart Failure measur,Average Mortality and Average Payment
2,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Payment for pneumonia patients,Greater Than the National Average Payment,536,"$20,216",MORT_PAYM_30_PN,Value of Care Pneumonia measure,Average Mortality and Higher Payment
3,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Payment for hip/knee replacement patients,Greater Than the National Average Payment,245,"$22,216",COMP_PAYM_90_HIP_KNEE,Value of Care hip/knee replacement,Average Complications and Higher Payment
4,10005,MARSHALL MEDICAL CENTERS,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,Payment for heart attack patients,No Different Than the National Average Payment,71,"$24,742",MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18835,670136,BAYLOR SCOTT & WHITE MEDICAL CENTER- AUSTIN,5245 W US 290,AUSTIN,TX,78735,Payment for hip/knee replacement patients,Not Available,Not Available,Not Available,COMP_PAYM_90_HIP_KNEE,Value of Care hip/knee replacement,Not Available
18836,670143,ASCENSION SETON BASTROP,"630 HIGWAY 71 W,",BASTROP,TX,78602,Payment for heart attack patients,Not Available,Not Available,Not Available,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Not Available
18837,670143,ASCENSION SETON BASTROP,"630 HIGWAY 71 W,",BASTROP,TX,78602,Payment for heart failure patients,Not Available,Not Available,Not Available,MORT_PAYM_30_HF,Value of Care Heart Failure measur,Not Available
18838,670143,ASCENSION SETON BASTROP,"630 HIGWAY 71 W,",BASTROP,TX,78602,Payment for pneumonia patients,Not Available,Not Available,Not Available,MORT_PAYM_30_PN,Value of Care Pneumonia measure,Not Available


In [21]:
no_nans_df = updated_df[updated_df.payment != 'Not Available']
no_nans_df

,facility_id,facility_name,address,city,state,zip_code,payment_measure_name,payment_category,denominator,payment,value_of_care_display_id,value_of_care_display_name,value_of_care_category
0,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Payment for heart attack patients,No Different Than the National Average Payment,574,"$24,934",MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment
1,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Payment for heart failure patients,No Different Than the National Average Payment,823,"$17,854",MORT_PAYM_30_HF,Value of Care Heart Failure measur,Average Mortality and Average Payment
2,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Payment for pneumonia patients,Greater Than the National Average Payment,536,"$20,216",MORT_PAYM_30_PN,Value of Care Pneumonia measure,Average Mortality and Higher Payment
3,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Payment for hip/knee replacement patients,Greater Than the National Average Payment,245,"$22,216",COMP_PAYM_90_HIP_KNEE,Value of Care hip/knee replacement,Average Complications and Higher Payment
4,10005,MARSHALL MEDICAL CENTERS,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,Payment for heart attack patients,No Different Than the National Average Payment,71,"$24,742",MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18786,670120,THE HOSPITALS OF PROVIDENCE TRANSMOUNTAIN CAMPUS,2000 TRANSMOUNTAIN RD,EL PASO,TX,79911,Payment for pneumonia patients,No Different Than the National Average Payment,143,"$20,221",MORT_PAYM_30_PN,Value of Care Pneumonia measure,Average Mortality and Average Payment
18792,670122,HOUSTON METHODIST THE WOODLANDS HOSPITAL,17201 INTERSTATE 45 SOUTH,THE WOODLANDS,TX,77385,Payment for heart attack patients,No Different Than the National Average Payment,53,"$27,138",MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment
18793,670122,HOUSTON METHODIST THE WOODLANDS HOSPITAL,17201 INTERSTATE 45 SOUTH,THE WOODLANDS,TX,77385,Payment for heart failure patients,Greater Than the National Average Payment,155,"$21,647",MORT_PAYM_30_HF,Value of Care Heart Failure measur,Average Mortality and Higher Payment
18794,670122,HOUSTON METHODIST THE WOODLANDS HOSPITAL,17201 INTERSTATE 45 SOUTH,THE WOODLANDS,TX,77385,Payment for pneumonia patients,No Different Than the National Average Payment,205,"$19,409",MORT_PAYM_30_PN,Value of Care Pneumonia measure,Better Mortality and Average Payment


In [22]:
no_nans_df.state.value_counts()

CA    912
TX    891
FL    625
IL    533
PA    500
NY    488
OH    483
MI    375
IN    338
NC    333
GA    333
WI    329
MO    295
VA    271
IA    259
TN    258
KY    248
KS    244
LA    241
OK    239
NJ    237
WA    230
AL    226
MA    214
MN    208
AZ    203
CO    200
MS    198
AR    189
SC    170
OR    170
MD    167
NE    165
WV    121
UT    110
ME    108
CT     99
NV     98
NM     93
MT     88
ID     88
NH     87
SD     82
ND     62
PR     59
WY     57
VT     46
HI     43
AK     36
RI     35
DE     24
DC     21
VI      6
MP      3
GU      3
Name: state, dtype: int64

In [23]:
no_nans_df.payment_measure_name.unique()

array(['Payment for heart attack patients',
       'Payment for heart failure patients',
       'Payment for pneumonia patients',
       'Payment for hip/knee replacement patients'], dtype=object)

In [24]:
no_nans_df.payment_category.unique()

array(['No Different Than the National Average Payment',
       'Greater Than the National Average Payment',
       'Less Than the National Average Payment'], dtype=object)

In [25]:
no_nans_df.value_of_care_category.unique()

array(['Average Mortality and Average Payment',
       'Average Mortality and Higher Payment',
       'Average Complications and Higher Payment',
       'Worse Mortality and Average Payment',
       'Worse Mortality and Lower Payment',
       'Average Complications and Average Payment',
       'Average Mortality and Lower Payment',
       'Average Complications and Lower Payment',
       'Better Mortality and Average Payment',
       'Worse Complications and Higher Payment',
       'Better Mortality and Lower Payment',
       'Worse Mortality and Higher Payment',
       'Worse Complications and Lower Payment',
       'Better Mortality and Higher Payment',
       'Better Complications and Lower Payment',
       'Better Complications and Average Payment',
       'Better Complications and Higher Payment',
       'Worse Complications and Average Payment', 'Not Available'],
      dtype=object)

In [26]:
clean_value_df=no_nans_df[no_nans_df["value_of_care_category"] != "Not Available"]
clean_value_df

,facility_id,facility_name,address,city,state,zip_code,payment_measure_name,payment_category,denominator,payment,value_of_care_display_id,value_of_care_display_name,value_of_care_category
0,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Payment for heart attack patients,No Different Than the National Average Payment,574,"$24,934",MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment
1,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Payment for heart failure patients,No Different Than the National Average Payment,823,"$17,854",MORT_PAYM_30_HF,Value of Care Heart Failure measur,Average Mortality and Average Payment
2,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Payment for pneumonia patients,Greater Than the National Average Payment,536,"$20,216",MORT_PAYM_30_PN,Value of Care Pneumonia measure,Average Mortality and Higher Payment
3,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Payment for hip/knee replacement patients,Greater Than the National Average Payment,245,"$22,216",COMP_PAYM_90_HIP_KNEE,Value of Care hip/knee replacement,Average Complications and Higher Payment
4,10005,MARSHALL MEDICAL CENTERS,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,Payment for heart attack patients,No Different Than the National Average Payment,71,"$24,742",MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18786,670120,THE HOSPITALS OF PROVIDENCE TRANSMOUNTAIN CAMPUS,2000 TRANSMOUNTAIN RD,EL PASO,TX,79911,Payment for pneumonia patients,No Different Than the National Average Payment,143,"$20,221",MORT_PAYM_30_PN,Value of Care Pneumonia measure,Average Mortality and Average Payment
18792,670122,HOUSTON METHODIST THE WOODLANDS HOSPITAL,17201 INTERSTATE 45 SOUTH,THE WOODLANDS,TX,77385,Payment for heart attack patients,No Different Than the National Average Payment,53,"$27,138",MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment
18793,670122,HOUSTON METHODIST THE WOODLANDS HOSPITAL,17201 INTERSTATE 45 SOUTH,THE WOODLANDS,TX,77385,Payment for heart failure patients,Greater Than the National Average Payment,155,"$21,647",MORT_PAYM_30_HF,Value of Care Heart Failure measur,Average Mortality and Higher Payment
18794,670122,HOUSTON METHODIST THE WOODLANDS HOSPITAL,17201 INTERSTATE 45 SOUTH,THE WOODLANDS,TX,77385,Payment for pneumonia patients,No Different Than the National Average Payment,205,"$19,409",MORT_PAYM_30_PN,Value of Care Pneumonia measure,Better Mortality and Average Payment


In [27]:
clean_value_df[clean_value_df.columns[9]] = clean_value_df[clean_value_df.columns[9]].replace('[\$,]', '', regex=True).astype(float)
clean_value_df

<ipython-input-27-d458df61f040>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_value_df[clean_value_df.columns[9]] = clean_value_df[clean_value_df.columns[9]].replace('[\$,]', '', regex=True).astype(float)


,facility_id,facility_name,address,city,state,zip_code,payment_measure_name,payment_category,denominator,payment,value_of_care_display_id,value_of_care_display_name,value_of_care_category
0,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Payment for heart attack patients,No Different Than the National Average Payment,574,24934.0,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment
1,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Payment for heart failure patients,No Different Than the National Average Payment,823,17854.0,MORT_PAYM_30_HF,Value of Care Heart Failure measur,Average Mortality and Average Payment
2,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Payment for pneumonia patients,Greater Than the National Average Payment,536,20216.0,MORT_PAYM_30_PN,Value of Care Pneumonia measure,Average Mortality and Higher Payment
3,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Payment for hip/knee replacement patients,Greater Than the National Average Payment,245,22216.0,COMP_PAYM_90_HIP_KNEE,Value of Care hip/knee replacement,Average Complications and Higher Payment
4,10005,MARSHALL MEDICAL CENTERS,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,Payment for heart attack patients,No Different Than the National Average Payment,71,24742.0,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18786,670120,THE HOSPITALS OF PROVIDENCE TRANSMOUNTAIN CAMPUS,2000 TRANSMOUNTAIN RD,EL PASO,TX,79911,Payment for pneumonia patients,No Different Than the National Average Payment,143,20221.0,MORT_PAYM_30_PN,Value of Care Pneumonia measure,Average Mortality and Average Payment
18792,670122,HOUSTON METHODIST THE WOODLANDS HOSPITAL,17201 INTERSTATE 45 SOUTH,THE WOODLANDS,TX,77385,Payment for heart attack patients,No Different Than the National Average Payment,53,27138.0,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment
18793,670122,HOUSTON METHODIST THE WOODLANDS HOSPITAL,17201 INTERSTATE 45 SOUTH,THE WOODLANDS,TX,77385,Payment for heart failure patients,Greater Than the National Average Payment,155,21647.0,MORT_PAYM_30_HF,Value of Care Heart Failure measur,Average Mortality and Higher Payment
18794,670122,HOUSTON METHODIST THE WOODLANDS HOSPITAL,17201 INTERSTATE 45 SOUTH,THE WOODLANDS,TX,77385,Payment for pneumonia patients,No Different Than the National Average Payment,205,19409.0,MORT_PAYM_30_PN,Value of Care Pneumonia measure,Better Mortality and Average Payment


In [28]:
clean_value_df['payment'].apply(type).value_counts()

<class 'float'>    12135
Name: payment, dtype: int64

In [29]:
clean_value_df['zip_code'].apply(type).value_counts()

<class 'int'>    12135
Name: zip_code, dtype: int64

In [30]:
clean_value_df['denominator'] = clean_value_df['denominator'].apply(lambda x: x.replace('$', '').replace(',', '')).astype(float)
clean_value_df.sample(5)

<ipython-input-30-4183c62c1256>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_value_df['denominator'] = clean_value_df['denominator'].apply(lambda x: x.replace('$', '').replace(',', '')).astype(float)


,facility_id,facility_name,address,city,state,zip_code,payment_measure_name,payment_category,denominator,payment,value_of_care_display_id,value_of_care_display_name,value_of_care_category
1846,50528,MEMORIAL HOSPITAL LOS BANOS,520 WEST I ST,LOS BANOS,CA,93635,Payment for pneumonia patients,No Different Than the National Average Payment,98.0,17730.0,MORT_PAYM_30_PN,Value of Care Pneumonia measure,Average Mortality and Average Payment
12101,341304,VIDANT BERTIE HOSPITAL,1403 SOUTH KINGS STREET,WINDSOR,NC,27983,Payment for heart failure patients,Less Than the National Average Payment,52.0,15119.0,MORT_PAYM_30_HF,Value of Care Heart Failure measur,Average Mortality and Lower Payment
16930,460047,ST MARK'S HOSPITAL,1200 EAST 3900 SOUTH,SALT LAKE CITY,UT,84124,Payment for pneumonia patients,No Different Than the National Average Payment,404.0,18159.0,MORT_PAYM_30_PN,Value of Care Pneumonia measure,Better Mortality and Average Payment
4503,140048,ADVOCATE TRINITY HOSPITAL,2320 E 93RD ST,CHICAGO,IL,60617,Payment for hip/knee replacement patients,Greater Than the National Average Payment,48.0,24069.0,COMP_PAYM_90_HIP_KNEE,Value of Care hip/knee replacement,Average Complications and Higher Payment
14811,420087,ROPER HOSPITAL,316 CALHOUN ST,CHARLESTON,SC,29401,Payment for hip/knee replacement patients,Less Than the National Average Payment,1227.0,19653.0,COMP_PAYM_90_HIP_KNEE,Value of Care hip/knee replacement,Average Complications and Lower Payment


In [31]:
clean_value_df['outcome'] = clean_value_df['value_of_care_display_id'].map(lambda v: v.split('_')[0])
clean_value_df

<ipython-input-31-317e12ed0950>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_value_df['outcome'] = clean_value_df['value_of_care_display_id'].map(lambda v: v.split('_')[0])


,facility_id,facility_name,address,city,state,zip_code,payment_measure_name,payment_category,denominator,payment,value_of_care_display_id,value_of_care_display_name,value_of_care_category,outcome
0,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Payment for heart attack patients,No Different Than the National Average Payment,574.0,24934.0,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment,MORT
1,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Payment for heart failure patients,No Different Than the National Average Payment,823.0,17854.0,MORT_PAYM_30_HF,Value of Care Heart Failure measur,Average Mortality and Average Payment,MORT
2,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Payment for pneumonia patients,Greater Than the National Average Payment,536.0,20216.0,MORT_PAYM_30_PN,Value of Care Pneumonia measure,Average Mortality and Higher Payment,MORT
3,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Payment for hip/knee replacement patients,Greater Than the National Average Payment,245.0,22216.0,COMP_PAYM_90_HIP_KNEE,Value of Care hip/knee replacement,Average Complications and Higher Payment,COMP
4,10005,MARSHALL MEDICAL CENTERS,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,Payment for heart attack patients,No Different Than the National Average Payment,71.0,24742.0,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment,MORT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18786,670120,THE HOSPITALS OF PROVIDENCE TRANSMOUNTAIN CAMPUS,2000 TRANSMOUNTAIN RD,EL PASO,TX,79911,Payment for pneumonia patients,No Different Than the National Average Payment,143.0,20221.0,MORT_PAYM_30_PN,Value of Care Pneumonia measure,Average Mortality and Average Payment,MORT
18792,670122,HOUSTON METHODIST THE WOODLANDS HOSPITAL,17201 INTERSTATE 45 SOUTH,THE WOODLANDS,TX,77385,Payment for heart attack patients,No Different Than the National Average Payment,53.0,27138.0,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment,MORT
18793,670122,HOUSTON METHODIST THE WOODLANDS HOSPITAL,17201 INTERSTATE 45 SOUTH,THE WOODLANDS,TX,77385,Payment for heart failure patients,Greater Than the National Average Payment,155.0,21647.0,MORT_PAYM_30_HF,Value of Care Heart Failure measur,Average Mortality and Higher Payment,MORT
18794,670122,HOUSTON METHODIST THE WOODLANDS HOSPITAL,17201 INTERSTATE 45 SOUTH,THE WOODLANDS,TX,77385,Payment for pneumonia patients,No Different Than the National Average Payment,205.0,19409.0,MORT_PAYM_30_PN,Value of Care Pneumonia measure,Better Mortality and Average Payment,MORT


In [32]:
clean_value_df['value_code'] = clean_value_df['value_of_care_display_id'].map(lambda v: v.split('_')[-1])
clean_value_df

<ipython-input-32-f0ca45d6bc9e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_value_df['value_code'] = clean_value_df['value_of_care_display_id'].map(lambda v: v.split('_')[-1])


,facility_id,facility_name,address,city,state,zip_code,payment_measure_name,payment_category,denominator,payment,value_of_care_display_id,value_of_care_display_name,value_of_care_category,outcome,value_code
0,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Payment for heart attack patients,No Different Than the National Average Payment,574.0,24934.0,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment,MORT,AMI
1,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Payment for heart failure patients,No Different Than the National Average Payment,823.0,17854.0,MORT_PAYM_30_HF,Value of Care Heart Failure measur,Average Mortality and Average Payment,MORT,HF
2,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Payment for pneumonia patients,Greater Than the National Average Payment,536.0,20216.0,MORT_PAYM_30_PN,Value of Care Pneumonia measure,Average Mortality and Higher Payment,MORT,PN
3,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Payment for hip/knee replacement patients,Greater Than the National Average Payment,245.0,22216.0,COMP_PAYM_90_HIP_KNEE,Value of Care hip/knee replacement,Average Complications and Higher Payment,COMP,KNEE
4,10005,MARSHALL MEDICAL CENTERS,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,Payment for heart attack patients,No Different Than the National Average Payment,71.0,24742.0,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment,MORT,AMI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18786,670120,THE HOSPITALS OF PROVIDENCE TRANSMOUNTAIN CAMPUS,2000 TRANSMOUNTAIN RD,EL PASO,TX,79911,Payment for pneumonia patients,No Different Than the National Average Payment,143.0,20221.0,MORT_PAYM_30_PN,Value of Care Pneumonia measure,Average Mortality and Average Payment,MORT,PN
18792,670122,HOUSTON METHODIST THE WOODLANDS HOSPITAL,17201 INTERSTATE 45 SOUTH,THE WOODLANDS,TX,77385,Payment for heart attack patients,No Different Than the National Average Payment,53.0,27138.0,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment,MORT,AMI
18793,670122,HOUSTON METHODIST THE WOODLANDS HOSPITAL,17201 INTERSTATE 45 SOUTH,THE WOODLANDS,TX,77385,Payment for heart failure patients,Greater Than the National Average Payment,155.0,21647.0,MORT_PAYM_30_HF,Value of Care Heart Failure measur,Average Mortality and Higher Payment,MORT,HF
18794,670122,HOUSTON METHODIST THE WOODLANDS HOSPITAL,17201 INTERSTATE 45 SOUTH,THE WOODLANDS,TX,77385,Payment for pneumonia patients,No Different Than the National Average Payment,205.0,19409.0,MORT_PAYM_30_PN,Value of Care Pneumonia measure,Better Mortality and Average Payment,MORT,PN


In [33]:
clean_value_df=clean_value_df.drop(['value_of_care_display_id'], axis=1).reset_index(drop=True)

In [34]:
clean_value_df[clean_value_df.columns[-1]] = clean_value_df[clean_value_df.columns[-1]].replace('KNEE', 'HIP_KNEE', regex=True)

In [35]:
clean_value_df.head()

,facility_id,facility_name,address,city,state,zip_code,payment_measure_name,payment_category,denominator,payment,value_of_care_display_name,value_of_care_category,outcome,value_code
0,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Payment for heart attack patients,No Different Than the National Average Payment,574.0,24934.0,Value of Care Heart Attack measure,Average Mortality and Average Payment,MORT,AMI
1,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Payment for heart failure patients,No Different Than the National Average Payment,823.0,17854.0,Value of Care Heart Failure measur,Average Mortality and Average Payment,MORT,HF
2,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Payment for pneumonia patients,Greater Than the National Average Payment,536.0,20216.0,Value of Care Pneumonia measure,Average Mortality and Higher Payment,MORT,PN
3,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Payment for hip/knee replacement patients,Greater Than the National Average Payment,245.0,22216.0,Value of Care hip/knee replacement,Average Complications and Higher Payment,COMP,HIP_KNEE
4,10005,MARSHALL MEDICAL CENTERS,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,Payment for heart attack patients,No Different Than the National Average Payment,71.0,24742.0,Value of Care Heart Attack measure,Average Mortality and Average Payment,MORT,AMI


In [36]:
clean_value_df.to_csv("output_data/clean_value_df.csv", index=False)

### CSV on readmission rates

In [37]:
readmissions_df = pd.read_csv("data/Readmissions_Reduction_Program.csv", low_memory=False)
updated_readmissions_df=readmissions_df.drop(['Footnote', 'Start Date', 'End Date', "State", "Predicted Readmission Rate", "Expected Readmission Rate"], axis=1).reset_index(drop=True)
updated_readmissions_df.columns = updated_readmissions_df.columns.str.replace(' ', '_').str.lower()
updated_readmissions_df

,facility_name,facility_id,measure_name,number_of_discharges,excess_readmission_ratio,number_of_readmissions
0,SOUTHEAST ALABAMA MEDICAL CENTER,10001,READM-30-HIP-KNEE-HRRP,258.0,1.1735,17
1,SOUTHEAST ALABAMA MEDICAL CENTER,10001,READM-30-CABG-HRRP,268.0,1.1868,41
2,SOUTHEAST ALABAMA MEDICAL CENTER,10001,READM-30-AMI-HRRP,620.0,1.0204,100
3,SOUTHEAST ALABAMA MEDICAL CENTER,10001,READM-30-HF-HRRP,1107.0,1.0702,264
4,SOUTHEAST ALABAMA MEDICAL CENTER,10001,READM-30-COPD-HRRP,444.0,1.0560,92
...,...,...,...,...,...,...
19219,ASCENSION SETON BASTROP,670143,READM-30-CABG-HRRP,NaN,NaN,NaN
19220,ASCENSION SETON BASTROP,670143,READM-30-COPD-HRRP,NaN,NaN,NaN
19221,ASCENSION SETON BASTROP,670143,READM-30-HF-HRRP,NaN,NaN,NaN
19222,ASCENSION SETON BASTROP,670143,READM-30-HIP-KNEE-HRRP,NaN,NaN,NaN


In [38]:
no_nans_readmissions=updated_readmissions_df.dropna()
no_nans_readmissions

,facility_name,facility_id,measure_name,number_of_discharges,excess_readmission_ratio,number_of_readmissions
0,SOUTHEAST ALABAMA MEDICAL CENTER,10001,READM-30-HIP-KNEE-HRRP,258.0,1.1735,17
1,SOUTHEAST ALABAMA MEDICAL CENTER,10001,READM-30-CABG-HRRP,268.0,1.1868,41
2,SOUTHEAST ALABAMA MEDICAL CENTER,10001,READM-30-AMI-HRRP,620.0,1.0204,100
3,SOUTHEAST ALABAMA MEDICAL CENTER,10001,READM-30-HF-HRRP,1107.0,1.0702,264
4,SOUTHEAST ALABAMA MEDICAL CENTER,10001,READM-30-COPD-HRRP,444.0,1.0560,92
...,...,...,...,...,...,...
19143,THE HOSPITALS OF PROVIDENCE TRANSMOUNTAIN CAMPUS,670120,READM-30-HF-HRRP,170.0,1.1127,47
19145,THE HOSPITALS OF PROVIDENCE TRANSMOUNTAIN CAMPUS,670120,READM-30-PN-HRRP,157.0,0.9826,22
19154,HOUSTON METHODIST THE WOODLANDS HOSPITAL,670122,READM-30-COPD-HRRP,88.0,0.9997,17
19155,HOUSTON METHODIST THE WOODLANDS HOSPITAL,670122,READM-30-HF-HRRP,204.0,0.9847,43


In [42]:
no_nans_readmissions['excess_readmission_ratio'].apply(type).value_counts()

<class 'float'>    11103
Name: excess_readmission_ratio, dtype: int64

In [43]:
no_nans_readmissions['number_of_discharges'].apply(type).value_counts()

<class 'float'>    11103
Name: number_of_discharges, dtype: int64

In [44]:
no_nans_readmissions['number_of_readmissions'].apply(type).value_counts()

<class 'str'>    11103
Name: number_of_readmissions, dtype: int64

In [46]:
no_nans_readmissions['number_of_readmissions'] = no_nans_readmissions['number_of_readmissions'].apply(pd.to_numeric)

<ipython-input-46-a49fa4ce6279>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_nans_readmissions['number_of_readmissions'] = no_nans_readmissions['number_of_readmissions'].apply(pd.to_numeric)


In [47]:
no_nans_readmissions['facility_id'].apply(type).value_counts()

<class 'int'>    11103
Name: facility_id, dtype: int64

In [48]:
no_nans_readmissions.describe()

,facility_id,number_of_discharges,excess_readmission_ratio,number_of_readmissions
count,11103.000000,11103.000000,11103.000000,11103.000000
mean,257786.415293,397.165541,1.007267,63.717644
std,155964.882032,365.180104,0.088375,63.022506
min,10001.000000,26.000000,0.611200,11.000000
25%,110083.000000,161.000000,0.956200,22.000000
50%,250042.000000,292.000000,1.003900,43.000000
75%,390041.000000,509.000000,1.053600,82.000000
max,670122.000000,9285.000000,2.337800,916.000000


In [49]:
no_nans_readmissions.to_csv("output_data/readmissions_df.csv", index=False)

### Export to PostgreSQL

In [41]:
from sqlalchemy import create_engine

In [68]:
final_census_df

,zip_code,population,poverty_rate,median_income
0,00601,17242.0,62.475351,13092.0
1,00602,38442.0,51.014515,16358.0
2,00603,48814.0,49.856599,16603.0
3,00606,6437.0,64.672984,12832.0
4,00610,27073.0,43.305138,19309.0
...,...,...,...,...
33113,99921,1986.0,14.400806,58571.0
33114,99922,330.0,39.090909,34028.0
33116,99925,927.0,18.554477,57375.0
33117,99926,1635.0,14.373089,53409.0
